In [1]:
#Ansatz: Der Nutzer gibt dem Programm ein beliebig langes Soundfile im wav-Format als Input.
#Von diesem wird die Länge durch die "soundfile-Library" ermittelt und entsprechend der Länge wird der Song anschließend mithilfe der "pydub-Library" in n 3-Sekunden-Abschnitte aufgeteilt.
#Für jeden dieser Abschnitte werden mithilfe der "librosa-library" (eventuell zusätzlich weitere) musikalische Eigenschaften, wie Tempo, oder Harmonieren isoliert und separiert gespeichert.
#Für jeden Abschnitt resultiert ein Array an isolierten Eigenschaften, aufgrund dessen ein fertig trainiertes ML-Modell predictions über das Gerne machen kann.
#Zuletzt wird das Genre als Output ausgewählt, welches für die meisten Abschnitte von dem ML-Algorithmus vorausgesagt wurde.

#Für die Entwicklung werden zwei Programme benötigt. In Programm 1 wird ein Neuronales Netz auf die korrekte Genrebestimmung trainiert.
#In Programm 2 wird das entwickelte Neuronale Netz für die Prediction eines eingegebenen Soundfiles genutzt

#Wir befinden uns in Programm 1. Als Basis des Training werden ein vorbereiteter Datensatz aus dem Internet uns selbst gelabelte Songs genutzt

In [2]:
#Import der benötigten Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [7]:
#Config
import os

soundLength = 3
os_dir = os.getcwd()


path_to_soundfilefolder = f'{os_dir}\\Datensatz\\genres_original'
print(path_to_soundfilefolder)

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original


In [8]:
#Import der Sound-Dateien\n
import os
filecount = 0
for dirname, _, filenames in os.walk(path_to_soundfilefolder):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        filecount += 1

    print(f"Anzahl der Trainingsdateien: {filecount}")

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original
Anzahl der Trainingsdateien: 0
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00000.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00006.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00007.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00008.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.00009.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\blues\blues.000

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00006.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00007.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00008.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00009.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00010.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00011.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_origina

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00096.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00097.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00098.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\classical\classical.00099.wav
Anzahl der Trainingsdateien: 200
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00000.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\cou

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00094.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00095.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00096.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00097.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00098.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\country\country.00099.wav
Anzahl der Trainingsdateien: 300
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00000.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00004.wav
C:\Users\Luis\Music_Gen

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00097.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00098.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\disco\disco.00099.wav
Anzahl der Trainingsdateien: 400
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00000.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00006.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00007.wav
C:\Users\Luis\Music_Genre_Git\

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00098.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\hiphop\hiphop.00099.wav
Anzahl der Trainingsdateien: 500
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00000.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00001.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00002.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00003.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00006.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00007.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\jazz.00008.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\jazz\j

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00004.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00005.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00006.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00007.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00008.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00009.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00010.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00011.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00012.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00013.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00014.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\metal.00015.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\metal\me

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00008.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00009.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00010.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00011.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00012.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00013.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00014.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00015.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00016.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00017.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00018.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00019.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\pop\pop.00020.wav
C:\Users\Luis\Music_Genre_Git\Datensat

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00016.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00017.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00018.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00019.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00020.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00021.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00022.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00023.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00024.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00025.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00026.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\reggae\reggae.00027.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\

C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00018.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00019.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00020.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00021.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00022.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00023.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00024.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00025.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00026.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00027.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00028.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00029.wav
C:\Users\Luis\Music_Genre_Git\Datensatz\genres_original\rock\rock.00030.wav
C:\Users\Lui

In [9]:
#BEISPIELCODE
#Ermittlung der Länge des Soundfiles und Anzahl der Slices (für eine Datei)
import soundfile as sf
import math
f = sf.SoundFile(f'{os_dir}\\Datensatz\\genres_original\\blues\\blues.00000.wav')
print('samples = {}'.format(len(f)))
print('sample rate = {}'.format(f.samplerate))
seconds = (len(f) / f.samplerate)
print(seconds)
slicecount=math.floor(seconds/soundLength)
print(slicecount)

samples = 661794
sample rate = 22050
30.013333333333332
10


In [11]:
##Benennung der Dateien ist {genre.filenumber}.wav -> Genre kann hierüber ermittelt werden
#slicenumber = 1

#Anwendung der Aufspaltung in 3-Sekunden-Abschnitte auf sämtliche Dateien
import os
import soundfile as sf
import math
from pydub import AudioSegment
from pathlib import Path 

#Create Folder if processed audio not exists
Path(f"{os_dir}\\Datensatz\\processed_audio").mkdir(parents=True, exist_ok=True)

filenumber = 0

for dirname, _, filenames in os.walk(path_to_soundfilefolder):
    for filename in filenames:
        filenumber +=1
        file = os.path.join(dirname, filename)
        f = sf.SoundFile(file)
        seconds = (len(f) / f.samplerate)
        slicecount=math.floor(seconds/soundLength)
        for slicenumber in range(slicecount):
            t1=slicenumber*soundLength*1000 #da Millisekunden
            t2=t1 + soundLength*1000 #hat die Länge in Sekunden von soundLength
            newAudio = AudioSegment.from_wav(file)
            newAudio = newAudio[t1:t2]
            genre = dirname.split("\\")[-1]
            newAudio.export(f'{os_dir}\\Datensatz\\processed_audio\\{genre}_{filenumber}_{slicenumber}.wav', format="wav") #Dateibenennung: Gerne_filenummer_slicenummer.wav